In [17]:
import pennylane as qml
from pennylane import numpy as np
from tensorflow import keras
from sklearn import decomposition
from pennylane.optimize import NesterovMomentumOptimizer

In [18]:
n_epochs = 30   # Number of optimization epochs
n_train = 60000    # Size of the train dataset
n_test = 10000     # Size of the test dataset
n_dim = 784        # 需要降到多少维
target_label_list = [0,1] #需要提取数据的标签列表

In [19]:
SAVE_PATH = "QNN/data/" # Data saving folder
PREPROCESS = True          # If False, skip processing and load data from SAVE_PATH
PCA_DR = False               # 是否进行PCA降维处理
select_samples_with_labels = True       # 是否挑选特定标签的数据

In [20]:
mnist_dataset = keras.datasets.mnist
(train_images, train_labels), (test_images, test_labels) = mnist_dataset.load_data()

# Reduce dataset size
train_images = train_images[:n_train]
train_labels = train_labels[:n_train]
test_images = test_images[:n_test]
test_labels = test_labels[:n_test]

# Normalize pixel values within 0 and 1
train_images = train_images / 255
test_images = test_images / 255

In [21]:
#做2分类两种方案，1、抽取0和1作为数据集， 2、将0-4作为第一类对应0，5-9作为第二类对应1
def extract_data_with_label(origin_data, origin_label, target_label_list):
    new_data = []
    new_labels = []
    for i in range(len(origin_label)):
        for j in range(len(target_label_list)):
            if(origin_label[i] == target_label_list[j]):
                new_data.append(origin_data[i,:,:])
                new_labels.append(origin_label[i])
    new_data = np.array(new_data)
    new_labels = np.array(new_labels)

    return new_data, new_labels
#print(type(test_labels))

In [22]:
if select_samples_with_labels == True:
    test_images, test_labels = extract_data_with_label(test_images, test_labels, target_label_list)
    train_images, train_labels = extract_data_with_label(train_images, train_labels, target_label_list) 

In [23]:
def pca_with_origin_data(ori_data, n_dim):
    #进行PCA降维
    n_sample, a, b = np.shape(ori_data)
    ori_data = ori_data.reshape(n_sample,a*b)
    pca = decomposition.PCA(n_components = n_dim)
    return pca.fit_transform(ori_data)

if PCA_DR == True: 
    test_images = pca_with_origin_data(test_images,n_dim)
    train_images = pca_with_origin_data(train_images,n_dim)

In [24]:
n_wires = int(np.ceil(np.log(n_dim) / np.log(2)))
dev = qml.device("default.qubit", wires=n_wires)

def layer(W):

    for i in range(n_wires):
        qml.Rot(W[i,0], W[i,1], W[i,2], wires=i)
        if i == n_wires - 1:
            qml.CNOT(wires = [i, 0])
        else:
            qml.CNOT(wires = [i, i+1])

@qml.qnode(dev)
def circuit(weights, f=False):
    #初态制备
    #qml.templates.AmplitudeEmbedding(features=f, wires=range(n_wires), normalize=True, pad_with=0.)
    qml.AmplitudeEmbedding(f, wires=range(n_wires), normalize=True, pad_with=0.)

    for W in weights:
        layer(W)

    return qml.expval(qml.PauliZ(0))

In [25]:
def variational_classifier(weights, bias, x):
    return circuit(weights, x) + bias

def square_loss(labels, predictions):
    loss = 0
    for l, p in zip(labels, predictions):
        loss = loss + (l - p) ** 2

    loss = loss / len(labels)
    return loss

def accuracy(labels, predictions):

    loss = 0
    for l, p in zip(labels, predictions):
        if abs(l - p) < 1e-5:
            loss = loss + 1
    loss = loss / len(labels)

    return loss

def cost(weights, bias, features, labels):
    predictions = [variational_classifier(weights, bias, f) for f in features]
    return square_loss(labels, predictions)


In [26]:
def process_img_to_features(image):
    img = image.flatten()
    return img

if PREPROCESS == True: #将图像数据拉成1维
    new_train_images = []
    print("pre-processing of train images:")
    for idx, img in enumerate(train_images):
        print("{}/{}        ".format(idx + 1, np.shape(train_images)[0]), end="\r")
        new_train_images.append(process_img_to_features(img))
      
    #q_train_images = np.asarray(q_train_images)
    train_images = np.array(new_train_images, requires_grad=False)

    new_test_images = []
    print("\npre-processing of test images:")
    for idx, img in enumerate(test_images):
        print("{}/{}        ".format(idx + 1, np.shape(test_images)[0]), end="\r")
        new_test_images.append(process_img_to_features(img))
    #q_test_images = np.asarray(q_test_images)
    test_images = np.array(new_test_images, requires_grad=False)

    # Save pre-processed images
    np.save(SAVE_PATH + "new_train_images_" + str(n_dim) + "_01.npy", train_images)
    np.save(SAVE_PATH + "new_test_images_" + str(n_dim) + "_01.npy", test_images)

pre-processing of train images:


4424/12665        

8990/12665        

12665/12665        
pre-processing of test images:


In [ ]:
train_images = np.load(SAVE_PATH + "new_train_images_" + str(n_dim) + "_01.npy")
test_images = np.load(SAVE_PATH + "new_test_images_" + str(n_dim) + "_01.npy")

train_labels = train_labels * 2 - np.ones(len(train_labels)) # shift label from {0, 1} to {-1, 1}
test_labels  =  test_labels * 2 - np.ones(len(test_labels))

#初始化参数
np.random.seed(0)
num_qubits = n_wires
num_layers = 6
weights_init = 0.01 * np.random.randn(num_layers, num_qubits, 3, requires_grad=True)
bias_init = np.array(0.0, requires_grad=True)

opt = NesterovMomentumOptimizer(0.01)
batch_size = 5

weights = weights_init
bias = bias_init

for it in range(n_epochs):

    # Update the weights by one optimizer step
    batch_index = np.random.randint(0, len(train_images), (batch_size,))
    X_batch = train_images[batch_index]
    Y_batch = train_labels[batch_index]

    weights, bias, _, _ = opt.step(cost, weights, bias, X_batch, Y_batch)
    
    # Compute predictions on train and validation set
    predictions_train = [np.sign(variational_classifier(weights, bias, f)) for f in train_images]
    predictions_val = [np.sign(variational_classifier(weights, bias, f)) for f in test_images]

    # Compute accuracy on train and validation set
    acc_train = accuracy(train_labels, predictions_train)
    acc_val = accuracy(test_labels, predictions_val)

    print(
        "Iter: {:5d} | Cost: {:0.7f} | Acc train: {:0.7f} | Acc validation: {:0.7f} "
        "".format(it + 1, cost(weights, bias, train_images, train_labels), acc_train, acc_val)
    )

C:\Users\admin\AppData\Roaming\Python\Python39\site-packages\autograd\numpy\numpy_wrapper.py:156: ComplexWarning: Casting complex values to real discards the imaginary part
  return A.astype(dtype, order, casting, subok, copy)
